### TO DO:

- correct the path for the pipeline
- correct the pathes


### Check the HOG

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage import data, exposure
from PIL import Image
from sklearn.svm import SVC #SVM classifier
from sklearn.decomposition import PCA

resize_factor = (188, 250)

In [2]:
train_img = sorted(os.listdir('../data/train_set/train_images/'))

In [3]:
HOGs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/train_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time))    

--- 123.63870596885681 seconds ---


In [4]:
labels = pd.read_csv('../data/train_set/label.csv')

In [5]:
HOGs[0].shape

(49329,)

In [6]:
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)

In [7]:
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0]

In [21]:
start_time = time.time()
pca = PCA(n_components=256)
reduced_features = pca.fit_transform(np.array(HOGs))
linear_SVM = SVC(kernel="linear", C=0.025)
linear_SVM.fit(reduced_features, emotion_idx)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 50.67025685310364 seconds ---


### Test the model

In [9]:
test_img = sorted(os.listdir('../data/train_set/test_images/'))

In [14]:
HOGs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs_test.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 31.130597829818726 seconds ---


In [15]:
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)

In [16]:
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0]

In [22]:
start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("--- %s seconds ---" % (time.time() - start_time)) 
print(accuracy)

--- 2.191887855529785 seconds ---
0.10179640718562874


In [23]:
## test train accuracy

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("--- %s seconds ---" % (time.time() - start_time)) 
print(accuracy)

--- 1.263164758682251 seconds ---
0.6453226613306653


### RandomForestClassifier

In [26]:
from sklearn.ensemble import RandomForestClassifier

start_time = time.time()
rfc = RandomForestClassifier(max_depth=10, n_estimators=100)
rfc.fit(reduced_features, emotion_idx)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 2.5524439811706543 seconds ---


In [27]:
start_time = time.time()
accuracy = rfc.score(reduced_test, test_emotion_idx)
print("--- %s seconds ---" % (time.time() - start_time)) 
print(accuracy)

--- 0.025160789489746094 seconds ---
0.033932135728542916


pca

accuracy train